In [12]:
from radarplot.CIKM import CIKM
import numpy as np
import random

N = 10000 # Total number of dataponts
n_samples = 15   # Number of samples

cikm = CIKM('../data/train.txt', '../data/train.index')
cikm_X = []
cikm_y = []

random.seed(1)
for i in random.sample(range(0, N), n_samples):
    r = cikm.getRadar(i, sorted=False, reversed=False)
    cikm_X.append(r.getLastStackFeatures(3))
    cikm_y.append(r.label)
    
cikm_X = np.array(cikm_X)

m, n = cikm_X.shape
cikm_y = np.array(cikm_y)#.reshape(-1, 1) # we want a the shape (m, 1)
print(cikm_X.shape)
print(cikm_y.shape)


(15, 122412)
(15,)


In [8]:
#from sklearn.preprocessing import StandardScaler
#scaler = StandardScaler()
#scaled_cikm_X = scaler.fit_transform(cikm_X)

def standarize(x):
    return (x - 70) / 70

standarize = np.vectorize(standarize)

scaled_cikm_X = standarize(cikm_X)
del(cikm_X)
scaled_cikm_X_plus_bias = np.c_[np.ones((m, 1)), scaled_cikm_X] # we want the sape (m, nfeatures + 1)
del(scaled_cikm_X)
print(scaled_cikm_X_plus_bias)

[[ 1.         -1.         -1.         ...,  0.61428571  0.7         0.61428571]
 [ 1.         -1.         -1.         ...,  0.4         0.4         0.35714286]
 [ 1.          0.74285714  0.74285714 ..., -1.         -1.         -1.        ]
 ..., 
 [ 1.         -1.         -1.         ..., -1.         -1.         -1.        ]
 [ 1.         -1.         -0.67142857 ...,  0.1         0.1         0.05714286]
 [ 1.         -1.         -1.         ..., -1.         -1.         -1.        ]]


In [9]:
import tensorflow as tf

tf.reset_default_graph()

n_epochs = 10000
learning_rate = 0.00001

X = tf.constant(scaled_cikm_X_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(cikm_y, dtype=tf.float32, name="y")
#X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
#y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
#optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
#training_op = optimizer.minimize(mse)
gradients = 2/m * tf.matmul(tf.transpose(X), error)
training_op = tf.assign(theta, theta - learning_rate * gradients)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        if epoch % 10 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())
        sess.run(training_op)
    
    best_theta = theta.eval()

print("Best theta:")
print(best_theta)

ValueError: Dimension 1 in both shapes must be equal, but are 1 and 15 for 'Assign' (op: 'Assign') with input shapes: [122413,1], [122413,15].

In [5]:
tf.reset_default_graph()

learning_rate = 0.00001

X = tf.placeholder(tf.float32, shape=(None, n + 1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
#optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
#training_op = optimizer.minimize(mse)
gradients = 2/m * tf.matmul(tf.transpose(X), error)
training_op = tf.assign(theta, theta - learning_rate * gradients)

init = tf.global_variables_initializer()

In [8]:
import numpy.random as rnd

def fetch_batch(epoch, batch_index, batch_size):
    rnd.seed(epoch * n_batches + batch_index)
    indices = rnd.randint(m, size=batch_size)
    print('epoch:{} index:{}'.format(epoch, batch_index))
    X_batch = scaled_cikm_X_plus_bias[indices]
    y_batch = cikm_y[indices]
    return X_batch, y_batch

n_epochs = 10000
batch_size = 100
n_batches = int(np.ceil(m / batch_size))
print('m:{}, n_batches:{}'.format(m, n_batches))

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
            #if epoch % 10 == 0:
             #   print("Epoch {} MSE = {}".format(epoch, mse.eval()))

    best_theta = theta.eval()
    
print("Best theta:")
print(best_theta)

m:150, n_batches:2
epoch:0 index:0
epoch:0 index:1
epoch:1 index:0
epoch:1 index:1
epoch:2 index:0
epoch:2 index:1
epoch:3 index:0
epoch:3 index:1
epoch:4 index:0
epoch:4 index:1
epoch:5 index:0
epoch:5 index:1
epoch:6 index:0
epoch:6 index:1
epoch:7 index:0
epoch:7 index:1
epoch:8 index:0
epoch:8 index:1
epoch:9 index:0
epoch:9 index:1
epoch:10 index:0
epoch:10 index:1
epoch:11 index:0
epoch:11 index:1
epoch:12 index:0
epoch:12 index:1
epoch:13 index:0
epoch:13 index:1
epoch:14 index:0
epoch:14 index:1
epoch:15 index:0
epoch:15 index:1
epoch:16 index:0
epoch:16 index:1
epoch:17 index:0
epoch:17 index:1
epoch:18 index:0
epoch:18 index:1
epoch:19 index:0
epoch:19 index:1
epoch:20 index:0
epoch:20 index:1
epoch:21 index:0
epoch:21 index:1
epoch:22 index:0
epoch:22 index:1
epoch:23 index:0
epoch:23 index:1
epoch:24 index:0
epoch:24 index:1
epoch:25 index:0
epoch:25 index:1
epoch:26 index:0
epoch:26 index:1
epoch:27 index:0
epoch:27 index:1
epoch:28 index:0
epoch:28 index:1
epoch:29 index:

epoch:234 index:0
epoch:234 index:1
epoch:235 index:0
epoch:235 index:1
epoch:236 index:0
epoch:236 index:1
epoch:237 index:0
epoch:237 index:1
epoch:238 index:0
epoch:238 index:1
epoch:239 index:0
epoch:239 index:1
epoch:240 index:0
epoch:240 index:1
epoch:241 index:0
epoch:241 index:1
epoch:242 index:0
epoch:242 index:1
epoch:243 index:0
epoch:243 index:1
epoch:244 index:0
epoch:244 index:1
epoch:245 index:0
epoch:245 index:1
epoch:246 index:0
epoch:246 index:1
epoch:247 index:0
epoch:247 index:1
epoch:248 index:0
epoch:248 index:1
epoch:249 index:0
epoch:249 index:1
epoch:250 index:0
epoch:250 index:1
epoch:251 index:0
epoch:251 index:1
epoch:252 index:0
epoch:252 index:1
epoch:253 index:0
epoch:253 index:1
epoch:254 index:0
epoch:254 index:1
epoch:255 index:0
epoch:255 index:1
epoch:256 index:0
epoch:256 index:1
epoch:257 index:0
epoch:257 index:1
epoch:258 index:0
epoch:258 index:1
epoch:259 index:0
epoch:259 index:1
epoch:260 index:0
epoch:260 index:1
epoch:261 index:0
epoch:261 

epoch:462 index:0
epoch:462 index:1
epoch:463 index:0
epoch:463 index:1
epoch:464 index:0
epoch:464 index:1
epoch:465 index:0
epoch:465 index:1
epoch:466 index:0
epoch:466 index:1
epoch:467 index:0
epoch:467 index:1
epoch:468 index:0
epoch:468 index:1
epoch:469 index:0
epoch:469 index:1
epoch:470 index:0
epoch:470 index:1
epoch:471 index:0
epoch:471 index:1
epoch:472 index:0
epoch:472 index:1
epoch:473 index:0
epoch:473 index:1
epoch:474 index:0
epoch:474 index:1
epoch:475 index:0
epoch:475 index:1
epoch:476 index:0
epoch:476 index:1
epoch:477 index:0
epoch:477 index:1
epoch:478 index:0
epoch:478 index:1
epoch:479 index:0
epoch:479 index:1
epoch:480 index:0
epoch:480 index:1
epoch:481 index:0
epoch:481 index:1
epoch:482 index:0
epoch:482 index:1
epoch:483 index:0
epoch:483 index:1
epoch:484 index:0
epoch:484 index:1
epoch:485 index:0
epoch:485 index:1
epoch:486 index:0
epoch:486 index:1
epoch:487 index:0
epoch:487 index:1
epoch:488 index:0
epoch:488 index:1
epoch:489 index:0
epoch:489 

KeyboardInterrupt: 